## Using R6

Learn how to define R6 classes, and to create R6 objects. You'll also learn about the structure of R6 classes, and how to separate the user interface from the implementation details.

### Specifying the Microwave Oven Class
To create R6 objects, you first have to create a class generator, sometimes known as a factory. These are created using the R6Class() function.

The first argument to R6Class() is the name of the class of the objects that are created. By convention, this is written in UpperCamelCase. Another argument to R6Class() is called private and holds the data fields for the object. This argument should be a list, with names for each of its elements.

Further arguments to R6Class() will be discussed in the coming exercises. The pattern for defining an object factory is as follows.

thing_factory <- R6Class(
  
  "Thing",
  
  private = list(a_field = "a value", another_field = 123)

)

In [3]:
# install.packages("R6")
library(R6)

# Define microwave_oven_factory
microwave_oven_factory <- R6Class(
  "MicrowaveOven",
  private = list(
    power_rating_watts = 800
  )
)


package 'R6' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Migue\AppData\Local\Temp\RtmpGqgoSL\downloaded_packages


Warning message:
"package 'R6' was built under R version 3.6.3"

### Making Microwave Ovens
To make an object, you create a factory, then call its new() method. Note that you don't need to define this method; all factories have a new() method by default.

In [4]:
# View the microwave_oven_factory
microwave_oven_factory

# Make a new microwave oven
microwave_oven <- microwave_oven_factory$new()

<MicrowaveOven> object generator
  Public:
    clone: function (deep = FALSE) 
  Private:
    power_rating_watts: 800
  Parent env: <environment: R_GlobalEnv>
  Locked objects: TRUE
  Locked class: FALSE
  Portable: TRUE

### Learning to Cook
The third argument to R6Class() is called public and holds the user-facing functionality for the object. This argument should be a list, with names for each of its elements.

The public element of an R6 class contains the functionality available to the user. Usually it will only contain functions.

The updated pattern for creating an R6 class generator is as follows:

thing_factory <- R6Class(
  
  "Thing",
  
  private = list( a_field = "a value", another_field = 123),
  
  public = list(
    do_something = function(x, y, z) {
      
      # Do something here
    
    }
    
  )

)

In [5]:
# Add a cook method to the factory definition
microwave_oven_factory <- R6Class(
  "MicrowaveOven",
  private = list(
    power_rating_watts = 800
  ),
  public = list(
    cook = function(time_seconds) {
      Sys.sleep(time_seconds)
      print("Your food is cooked!")
    }
  )
)

# Create microwave oven object
a_microwave_oven <- microwave_oven_factory$new()

# Call cook method for 1 second
a_microwave_oven$cook(1)

[1] "Your food is cooked!"


### Close the Door
Methods for an R6 object can access its private fields by using the private$ prefix.

In [6]:
# Add a close_door() method
microwave_oven_factory <- R6Class(
  "MicrowaveOven",
  private = list(
    power_rating_watts = 800,
    door_is_open = FALSE
  ),
  public = list(
    cook = function(time_seconds) {
      Sys.sleep(time_seconds)
      print("Your food is cooked!")
    },
    open_door = function() {
      private$door_is_open <- TRUE
    },
    close_door = function(){
      private$door_is_open <- FALSE
    }
  )
)

## First Thing's First
There is one special public method named initialize() (note the American English spelling). This is not called directly by the user. Instead, it is called automatically when an object is created; that is, when the user calls new().

initialize() lets you set the values of the private fields when you create an R6 object. The pattern for an initialize() function is as follows:

thing_factory <- R6Class(
  
    "Thing",
  
    private = list(a_field = "a value", another_field = 123),
    
    public = list(
    
    initialize = function(a_field, another_field) {
        
      if(!missing(a_field)) {
          
        private$a_field <- a_field
      
      }
      
        if(!missing(another_field)) {
            
        private$another_field <- another_field
      }
        
    }
        
  )

)

Notice the use of missing(). This returns TRUE if an argument wasn't passed in the function call.

Arguments to the factory's new() method are passed to initialize().

a_thing <- thing_factory$new(
    
    a_field = "a different value", 
  
    another_field = 456
)

In [7]:
# Add an initialize method
microwave_oven_factory <- R6Class(
  "MicrowaveOven",
  private = list(
    power_rating_watts = 800,
    door_is_open = FALSE
  ),
  public = list(
    cook = function(time_seconds) {
      Sys.sleep(time_seconds)
      print("Your food is cooked!")
    },
    open_door = function() {
      private$door_is_open <- TRUE
    },
    close_door = function() {
      private$door_is_open <- FALSE
    },
    # Add initialize() method here
    initialize = function(power_rating_watts,door_is_open) {
      if(!missing(power_rating_watts)) {
        private$power_rating_watts <- power_rating_watts
      }
      if(!missing(door_is_open)) {
        private$door_is_open <- door_is_open
      }  
    }
  )
)

# Make a microwave
a_microwave_oven <- microwave_oven_factory$new(650, TRUE)


### Read the Rating
The data stored by an R6 object is deliberately hidden away from the user by keeping it in the private element. This is the principle of encapsulation.

If you want to provide access to any of the data values, you can use an active binding. These are functions that behave like variables.

Active bindings are stored in the active element of an R6 object. To create an active binding to get a private data field (i.e. a "read-only" binding), you create a function with no arguments that simply returns the private element.

The pattern for creating a read-only active binding is as follows:

thing_factory <- R6Class(
  
    "Thing",
  
    private = list(
    
        ..a_field = "a value"
  ),
  
    active = list(
    a_field = function() {
        
      private$..a_field
    
    }
        
    )

)

The active binding is called like a data variable, not a function.

a_thing <- thing_factory$new()


a_thing$a_field

'#not a_thing$a_field()


In [9]:
# Add a binding for power rating
microwave_oven_factory <- R6Class(
  "MicrowaveOven",
  private = list(
    ..power_rating_watts = 800
  ),
  active = list(
    # Add the binding here
     power_rating_watts = function(){
       private$..power_rating_watts
     }
  )
)

# Make a microwave 
a_microwave_oven <- microwave_oven_factory$new()

# Get the power rating
a_microwave_oven$power_rating_watts

[1] 800

Control the Power
Active bindings can also be used to set private fields. In this case, the binding function should accept a single argument, named "value".

The pattern for creating a read/write active binding is as follows.

thing_factory <- R6Class(
    
    "Thing",
  
    private = list(..a_field = "a value"),
  
    active = list(
    a_field = function(value) {
      
        if(missing(value)) {
        
            private$..a_field
      } else {
        
            assert_is_a_string(value) # or another assertion
        
            private$..a_field <- value
      }
    }
  
    )

)

Values are assigned as though the binding was a data variable, not a function.

a_thing <- thing_factory$new()

a_thing$a_field <- "a new value"

'# not a_thing$a_field("a new value")

In [13]:
# install.packages("assertive")
library(assertive)

# Add a binding for power rating
microwave_oven_factory <- R6Class(
  "MicrowaveOven",
  private = list(
    ..power_rating_watts = 800,
    ..power_level_watts = 800
  ),
  # Add active list containing an active binding
  active = list(
    power_level_watts = function(value) {
      if(missing(value)) {
        # Return the private value
        private$..power_level_watts
      } else {
        # Assert that value is a number
        assert_is_a_number(value)
        # Assert that value is in a closed range from 0 to power rating
        assert_all_are_in_closed_range(value, 0,private$..power_rating_watts)
        # Set the private power level to value
        private$..power_level_watts <- value
      }
    }
  )
)

# Make a microwave 
a_microwave_oven <- microwave_oven_factory$new()

# Get the power level
a_microwave_oven$power_level_watts

# Try to set the power level to 1600 watts
a_microwave_oven$power_level_watts <- 1600

[1] 800

ERROR: Error in (function (value) : is_in_closed_range : value are not all in the range [0,800].
There was 1 failure:
  Position Value    Cause
1        1  1600 too high


In [14]:
# Set the power level to 400 watts
a_microwave_oven$power_level_watts <- 400

In [15]:
# Try to set the power level to "400"
a_microwave_oven$power_level_watts <- "400"

ERROR: Error in (function (value) : is_a_number : value is not of class 'numeric'; it has class 'character'.
